In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow import keras 
from keras.applications import inception_resnet_v2
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Sequential
from keras.utils import to_categorical 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import seaborn as sns 
import glob 
import cv2
import pathlib

In [2]:
dataset = pathlib.Path('lung_colon_image_set/colon_image_sets/')
images = list(dataset.glob('*/*.jpeg'))

cacc = dataset.glob('colon_aca/*')
cbt = dataset.glob('colon_n/*')

colon_adenocarcinoma = list(cacc)
colon_benign_tissue = list(cbt)

In [3]:
label_dict = {
    'colon_adenocarcinoma' : 0,
    'colon_benign_tissue' : 1
}

image_dict = {
    'colon_adenocarcinoma' : colon_adenocarcinoma,
    'colon_benign_tissue' :colon_benign_tissue
}

In [4]:
x,y = [],[]

for name,images in image_dict.items():
    for i in images:
        img = cv2.imread(str(i))
        re_img = cv2.resize(img,(200,200))
        re_img=np.round((re_img.astype(np.float16)/255.0),4)
        x.append(re_img)
        y.append(label_dict[name])

x = np.array(x)
y = np.array(y)

In [5]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=100)
x_train.shape

(7000, 200, 200, 3)

In [8]:
model = Sequential()
inc_res_model = inception_resnet_v2.InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(200,200,3))
model.add(inc_res_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(2,activation='softmax'))

for layer in inc_res_model.layers:
    layer.trainable = False

y_train_cat = to_categorical(y_train, num_classes=2)
y_test_cat = to_categorical(y_test,num_classes=2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Funct  (None, 4, 4, 1536)        54336736  
 ional)                                                          
                                                                 
 global_average_pooling2d_1  (None, 1536)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 2)                 3074      
                                                                 
Total params: 54339810 (207.29 MB)
Trainable params: 3074 (12.01 KB)
Non-trainable params: 54336736 (207.28 MB)
_________________________________________________________________


In [9]:
history = model.fit(x_train,y_train_cat,epochs=10,batch_size=8)

Epoch 1/10
875/875 [==============================] - 442s 490ms/step - loss: 0.0616 - accuracy: 0.9779
Epoch 2/10
186/875 [=====>........................] - ETA: 5:25 - loss: 0.0222 - accuracy: 0.9913

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
model.evaluate(x_test,y_test_cat)

In [ ]:
plt.plot(history.history['accuracy'],color='blue',label='accuracy')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
x=list(range(0,3))
plt.plot(hist['loss'])
plt.xticks(x)
plt.title("Training Loss")
plt.show()

In [ ]:
y_pred_model=model.predict(x_test)
y_pred_model=np.argmax(y_pred_model,axis=1)
print(classification_report(y_pred_model,y_test))
sns.heatmap(confusion_matrix(y_test,y_pred_model),annot=True,fmt='d')